<a href="https://colab.research.google.com/github/eyaler/workshop/blob/master/nn_8_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer shows why:

## Visualizing machine learning and decision

### Dr. Eyal Gruss

based on: https://github.com/raghakot/keras-vis/blob/master/examples/vggnet/attention.ipynb

slides: https://j.mp/vis-dl

In [ ]:
%tensorflow_version 1.x

# install keras-vis master version
!pip install -U git+https://github.com/raghakot/keras-vis.git
!pip install h5py==2.10.0

In [ ]:
# download and display some images
from vis.utils import utils
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (18, 24)

urls = ['https://i.imgur.com/raG3QsN.jpg', 'https://i.imgur.com/DGnn1nt.jpg', 'https://i.imgur.com/tQMznEK.png']

imgs = [utils.load_img(url, target_size=(224, 224))[..., :3] for url in urls]

f, ax = plt.subplots(1, len(imgs))
for i in range(len(imgs)):
  ax[i].imshow(imgs[i])


In [ ]:
# download imagenet recognition model and predict on images

from keras.applications.vgg16 import VGG16 as MODEL, preprocess_input, decode_predictions
#from keras.applications.resnet50 import ResNet50 as MODEL, preprocess_input, decode_predictions
import json
import numpy as np
plt.rcParams['axes.labelsize'] = 15

top = 5
min_prob = 0.01

!wget -N https://raw.githubusercontent.com/raghakot/keras-vis/master/resources/imagenet_class_index.json
with open('imagenet_class_index.json') as f:
    CLASS_INDEX = json.load(f)
label2index = {CLASS_INDEX[i][1]:int(i) for i in CLASS_INDEX}

model = MODEL(weights='imagenet')
preproc_imgs = [preprocess_input(img) for img in imgs]
probs = [model.predict(np.expand_dims(img, axis=0)) for img in preproc_imgs]
preds = [decode_predictions(prob, top=top)[0] for prob in probs]
all_preds = [{x[1]:x[2] for x in decode_predictions(prob,top=1000)[0]} for prob in probs]

# change final activation from softmax to linear for viz
from keras import activations
layer_index = -1
model.layers[layer_index].activation = activations.linear
model_linear = utils.apply_modifications(model)

f, ax = plt.subplots(1, len(imgs))
for i in range(len(imgs)):
  ax[i].imshow(imgs[i])
  ax[i].set_xlabel('\n'.join('\n%s (%d): %d%%'%(pred[1], label2index[pred[1]], pred[2]*100) for pred in preds[i] if pred[2]>min_prob))

In [ ]:
from vis.visualization import visualize_saliency, visualize_cam
import tensorflow as tf
import matplotlib.cm as cm
import cv2
from IPython.display import Javascript

def viz(filter_indices, method='sal', sal_backprop_modifier='guided', cam_backprop_modifier=None, sal_grad_modifier='absolute', cam_grad_modifier=None, layer_index=-1, penultimate_layer_idx=None, quantile=0.99, alpha=0.7, local_alpha=True, linear=True):
  display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))
  if linear:
    model1 = model_linear
  else:
    model1 = model

  f, ax = plt.subplots(3 if method=='cam' else 5, len(imgs))
  if method=='cam':
    f.set_figheight(12)
  for i in range(len(imgs)):
    grads = 1
    cam = 1
    with tf.compat.v1.get_default_graph().gradient_override_map({'LeakyRelu': 'guided'}):
      if method in ('sal','sal*cam'):
        grads = visualize_saliency(model1, layer_index, filter_indices, preproc_imgs[i], backprop_modifier=sal_backprop_modifier, grad_modifier=sal_grad_modifier)
      if method in ('cam','sal*cam'):
        cam = visualize_cam(model1, layer_index, filter_indices, preproc_imgs[i], backprop_modifier=cam_backprop_modifier, grad_modifier=cam_grad_modifier, penultimate_layer_idx=None)

    ax[0,i].imshow(grads*cam, cmap='jet')
    if method!='cam':
      if quantile:
        grads = np.clip(grads,0,np.quantile(grads, quantile))
      ax[1,i].imshow(grads*cam, cmap='jet')

      grads = cv2.GaussianBlur(grads, (9,9), 0)
      ax[2,i].imshow(grads*cam, cmap='jet')

    grads *= cam
    if grads.max()>grads.min():
      grads = (grads-grads.min())/(grads.max()-grads.min())

    heatmap = cm.jet(grads)[...,:3]
    gray = imgs[i].mean(axis=-1, keepdims=True)/255
    alpha_filter = alpha
    if local_alpha:
      alpha_filter = alpha*alpha
    ax_num = 1 if method=='cam' else 3
    ax[ax_num ,i].imshow(alpha_filter*heatmap+(1-alpha_filter)*gray)
    if local_alpha:
      alpha_filter = alpha*np.expand_dims(grads, axis=-1)
      ax_num+=1
      ax[ax_num ,i].imshow(alpha_filter*heatmap+(1-alpha_filter)*gray)
    if isinstance(filter_indices,int):
      ax[ax_num, i].set_xlabel('\nprob=%.6f'%probs[i][0][filter_indices])

In [ ]:
# simple gradients - not so good
viz(filter_indices=label2index['Border_collie'], sal_backprop_modifier=None)

#Guided Backprop (https://arxiv.org/abs/1412.6806)

In [ ]:
viz(filter_indices=label2index['Border_collie'])

In [ ]:
#Note backprop is not descriminative between different output labels:
viz(filter_indices=label2index['space_shuttle'])

#Grad-CAM (https://arxiv.org/abs/1610.02391)

In [ ]:
viz(filter_indices=label2index['Border_collie'], method='cam')

In [ ]:
viz(filter_indices=label2index['ram'], method='cam')

In [ ]:
#we can see that actually Irish_wolfhound was found for the wrong reasons... but maybe it's acceptible (see image below)
viz(filter_indices=label2index['Irish_wolfhound'], method='cam')

#da real Irish wolfhound!


In [ ]:
#@title ![da real Irish wolfhound](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Irish_wolfhound_giaccomo.JPG/1024px-Irish_wolfhound_giaccomo.JPG)

#Disclaimer: Garbage in garbage out

In [ ]:
#note that keras-vis always normalizes (stretches) the grad-cam weights as well as the gradients, so we may always get something (https://github.com/raghakot/keras-vis/issues/178)
viz(filter_indices=label2index['space_shuttle'], method='cam')

In [ ]:
viz(filter_indices=label2index['dishwasher'], method='cam')

#Nagetive Grad-CAM

In [ ]:
#negate - highlight areas that decrease the class output
viz(filter_indices=label2index['Border_collie'], method='cam', cam_grad_modifier='negate')

In [ ]:
viz(filter_indices=label2index['ram'], method='cam', cam_grad_modifier='negate')

In [ ]:
viz(filter_indices=label2index['space_shuttle'], method='cam', cam_grad_modifier='negate')

#Guided Grad-CAM = Guided Backprop * Grad-CAM (Grad-CAM paper)

In [ ]:
#note this is different from keras-vis grad-cam with modifier="guided" (https://github.com/raghakot/keras-vis/issues/201)
viz(filter_indices=label2index['Border_collie'], method='sal*cam')

In [ ]:
viz(filter_indices=label2index['ram'], method='sal*cam')